In [1]:
import nfl_data_py as nfl
import pandas as pd
import sqlite3 as sq
pd.set_option('display.max_columns',30)

In [19]:
conn = sq.connect('data/nfl.db')

In [3]:
years=range(2000,2025)

In [4]:
weekly = nfl.import_weekly_data(years)
weekly.to_parquet('data/weekly.parquet')

Downcasting floats.


In [5]:
weekly.to_sql('weekly', conn, if_exists='replace', index=False)

129182

In [6]:
conn.execute('create index week_index on weekly(week)')
conn.execute('create index season_index on weekly(season)')

In [7]:
query = 'select * from weekly where position = "RB" and recent_team = "IND"'
result_df = pd.read_sql_query(query, conn)
result_df

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,...,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0000003,None,Abdul-Karim al-Jabbar,RB,RB,None,IND,2000,1,REG,KC,0,0,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,-0.2,-0.2
1,00-0006155,None,Lennox Gordon,RB,RB,None,IND,2000,15,REG,BUF,0,0,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.6,0.6
2,00-0006155,None,Lennox Gordon,RB,RB,None,IND,2000,17,REG,MIN,0,0,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.1,0.1
3,00-0008241,None,Edgerrin James,RB,RB,None,IND,2000,1,REG,KC,0,0,0.0,0,...,1,0.0,0.0,0.0,0.0,1.0,2.409343,0,0.00,0.206897,NaN,NaN,0.0,26.1,31.1
4,00-0008241,None,Edgerrin James,RB,RB,None,IND,2000,2,REG,LV,0,0,0.0,0,...,1,0.0,0.0,0.0,0.0,2.0,1.015774,0,0.00,0.148936,NaN,NaN,0.0,24.1,30.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,00-0037120,T.Goodson,Tyler Goodson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,IND,2024,8,REG,HOU,0,0,0.0,0,...,0,0.0,0.0,25.0,0.0,0.0,-0.593228,0,0.00,0.032258,0.051125,0.084174,0.0,0.0,0.0
969,00-0037120,T.Goodson,Tyler Goodson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,IND,2024,10,REG,BUF,0,0,0.0,0,...,1,0.0,0.0,12.0,3.0,1.0,1.822438,0,0.75,0.085714,0.043478,0.159006,0.0,6.9,8.9
970,00-0037120,T.Goodson,Tyler Goodson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,IND,2024,15,REG,DEN,0,0,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.3,0.3
971,00-0037120,T.Goodson,Tyler Goodson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,IND,2024,16,REG,TEN,0,0,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,2.2,2.2


In [8]:
pbp = nfl.import_pbp_data(years)

2000 done.
2001 done.
2002 done.
2003 done.
2004 done.
2005 done.
2006 done.
2007 done.
2008 done.
2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
Downcasting floats.


In [9]:
pbp.to_parquet('data/pbp.parquet')

In [10]:
pbp.to_sql('pbp', conn, if_exists='replace', index=False)

1182501

In [11]:
conn.execute('create index date_index on pbp(game_date)')
#conn.execute('create index season_index on pbp(season)')

In [12]:
rosters = nfl.import_weekly_rosters(range(2002,2025))
rosters

,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,height,weight,college,player_id,espn_id,...,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,age
0,2002,CHI,RB,None,27,ACT,Rabih Abdullah,Rabih,Abdullah,1975-04-27,72.0,220.0,None,00-0000007,None,...,4.0,https://static.www.nfl.com/image/private/f_aut...,None,14,REG,A01,Rabih,ABD675101,None,32004142-4467-5101-8326-4b71f9f0a7b7,1998.0,1998.0,None,None,27.619
1,2002,CHI,RB,None,27,ACT,Rabih Abdullah,Rabih,Abdullah,1975-04-27,72.0,220.0,None,00-0000007,None,...,4.0,https://static.www.nfl.com/image/private/f_aut...,None,5,REG,A01,Rabih,ABD675101,None,32004142-4467-5101-8326-4b71f9f0a7b7,1998.0,1998.0,None,None,27.447
2,2002,CHI,RB,None,27,ACT,Rabih Abdullah,Rabih,Abdullah,1975-04-27,72.0,220.0,None,00-0000007,None,...,4.0,https://static.www.nfl.com/image/private/f_aut...,None,13,REG,A01,Rabih,ABD675101,None,32004142-4467-5101-8326-4b71f9f0a7b7,1998.0,1998.0,None,None,27.598
3,2002,CHI,RB,None,27,ACT,Rabih Abdullah,Rabih,Abdullah,1975-04-27,72.0,220.0,None,00-0000007,None,...,4.0,https://static.www.nfl.com/image/private/f_aut...,None,2,REG,A01,Rabih,ABD675101,None,32004142-4467-5101-8326-4b71f9f0a7b7,1998.0,1998.0,None,None,27.387
4,2002,CHI,RB,None,27,ACT,Rabih Abdullah,Rabih,Abdullah,1975-04-27,72.0,220.0,None,00-0000007,None,...,4.0,https://static.www.nfl.com/image/private/f_aut...,None,16,REG,A01,Rabih,ABD675101,None,32004142-4467-5101-8326-4b71f9f0a7b7,1998.0,1998.0,None,None,27.655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858363,2024,IND,OL,T,NaN,DEV,Jack Wilson,Jack,Wilson,NaT,NaN,338.0,None,None,None,...,0.0,None,None,6,REG,P01,Jack,WIL715047,58027,None,2024.0,2024.0,None,NaN,NaN
858364,2024,NO,DB,DB,NaN,DEV,Tra Fluellen,Tra,Fluellen,NaT,NaN,210.0,None,None,None,...,0.0,None,None,17,REG,P01,Tra,FLU097161,58115,None,2024.0,2024.0,None,NaN,NaN
858365,2024,NO,DB,DB,NaN,DEV,Tra Fluellen,Tra,Fluellen,NaT,NaN,210.0,None,None,None,...,0.0,None,None,18,REG,P01,Tra,FLU097161,58115,None,2024.0,2024.0,None,NaN,NaN
858366,2024,NO,DB,DB,NaN,DEV,Tra Fluellen,Tra,Fluellen,NaT,NaN,210.0,None,None,None,...,0.0,None,None,16,REG,P01,Tra,FLU097161,58115,None,2024.0,2024.0,None,NaN,NaN


In [13]:
rosters.to_sql('rosters', conn, if_exists='replace', index=False)

858368

In [14]:
ids = nfl.import_ids()

In [15]:
ids.to_sql('ids', conn, if_exists='replace', index=False)

11777

In [16]:
snaps = nfl.import_snap_counts(range(2012,2025))
team_map = {'OAK':'LV',
            'STL':'LA',
            'SD':'LAC'}
snaps['team'] = snaps['team'].apply(lambda x: team_map.get(x,x))
snaps.to_sql('snaps', conn, if_exists='replace', index=False)

320424

In [20]:
depth = nfl.import_depth_charts(range(2001,2025)).drop_duplicates().reset_index(drop=True)

team_map = {'OAK':'LV',
            'STL':'LA',
            'SD':'LAC'}
depth['club_code'] = depth['club_code'].apply(lambda x: team_map.get(x,x))

depth = depth.sort_values(by=['season','club_code','week','depth_team'])
depth['dense_depth'] = depth.groupby(['club_code','season','week','position', 'game_type','formation']).cumcount()
depth['depth_position'] = depth['depth_position'].replace({'\n    ':None})
depth['depth_position'] = depth['depth_position'].fillna(depth['position'])
depth = depth.rename({'club_code':'team_name'},axis=1)
depth.to_sql('depth', conn, if_exists='replace', index=False)

863061

In [21]:
schedule = nfl.import_schedules(years)
schedule.to_sql('schedule', conn, if_exists='replace', index=False)
schedule

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,...,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
259,2000_01_SF_ATL,2000,REG,1,2000-09-03,Sunday,13:00,SF,28.0,ATL,36.0,Home,8.0,64.0,0.0,...,NaN,1,dome,astroturf,NaN,NaN,00-0005755,00-0002876,Jeff Garcia,Chris Chandler,Steve Mariucci,Dan Reeves,Walt Coleman,ATL00,Georgia Dome
260,2000_01_JAX_CLE,2000,REG,1,2000-09-03,Sunday,13:00,JAX,27.0,CLE,7.0,Home,-20.0,34.0,0.0,...,NaN,1,outdoors,grass,78.0,6.0,00-0002110,00-0003535,Mark Brunell,Tim Couch,Tom Coughlin,Chris Palmer,Bernie Kukar,CLE00,Cleveland Browns Stadium
261,2000_01_IND_KC,2000,REG,1,2000-09-03,Sunday,13:00,IND,27.0,KC,14.0,Home,-13.0,41.0,0.0,...,NaN,0,outdoors,grass,90.0,5.0,00-0010346,00-0006300,Peyton Manning,Elvis Grbac,Jim Mora,Gunther Cunningham,Larry Nemmers,KAN00,Arrowhead Stadium
262,2000_01_CHI_MIN,2000,REG,1,2000-09-03,Sunday,13:00,CHI,27.0,MIN,30.0,Home,3.0,57.0,0.0,...,NaN,1,dome,astroturf,NaN,NaN,00-0011044,00-0003739,Cade McNown,Daunte Culpepper,Dick Jauron,Dennis Green,Ed Hochuli,MIN00,Hubert H. Humphrey Metrodome
263,2000_01_TB_NE,2000,REG,1,2000-09-03,Sunday,13:00,TB,21.0,NE,16.0,Home,-5.0,37.0,0.0,...,NaN,0,outdoors,grass,71.0,5.0,00-0009266,00-0001361,Shaun King,Drew Bledsoe,Tony Dungy,Bill Belichick,Jeff Triplette,BOS99,Foxboro Stadium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6979,2024_19_PIT_BAL,2024,WC,19,2025-01-11,Saturday,20:00,PIT,NaN,BAL,NaN,Home,NaN,NaN,NaN,...,-115.0,1,outdoors,grass,NaN,NaN,00-0029263,00-0034796,Russell Wilson,Lamar Jackson,Mike Tomlin,John Harbaugh,NaN,BAL00,M&T Bank Stadium
6980,2024_19_DEN_BUF,2024,WC,19,2025-01-12,Sunday,13:00,DEN,NaN,BUF,NaN,Home,NaN,NaN,NaN,...,-118.0,0,outdoors,a_turf,NaN,NaN,00-0039732,00-0034857,Bo Nix,Josh Allen,Sean Payton,Sean McDermott,NaN,BUF00,New Era Field
6981,2024_19_GB_PHI,2024,WC,19,2025-01-12,Sunday,16:30,GB,NaN,PHI,NaN,Home,NaN,NaN,NaN,...,-105.0,0,outdoors,NaN,NaN,NaN,00-0036264,00-0036389,Jordan Love,Jalen Hurts,Matt LaFleur,Nick Sirianni,NaN,PHI00,Lincoln Financial Field
6982,2024_19_WAS_TB,2024,WC,19,2025-01-12,Sunday,20:00,WAS,NaN,TB,NaN,Home,NaN,NaN,NaN,...,-112.0,0,outdoors,NaN,NaN,NaN,00-0039910,00-0034855,Jayden Daniels,Baker Mayfield,Dan Quinn,Todd Bowles,NaN,TAM00,Raymond James Stadium


In [22]:
conn.commit()
conn.close()